In [2]:
from IPython.display import Image
from IPython.core.display import HTML 

## What was done in the previous notebook:

- intro to Django
- components of a Django application
- creating a project and a blog app
- urls and views
- connecting our app urls with project urls
- creating templates foor our home and about webpages
- creating models for posts and comments
- migrations (kind of like git commit)
- Bootstrap
- incorporating posts into views
- making a post template
- django admin

Let's first check where we left off with our site.
Reopen the project and the folder. Restart your virtual environment by typing:

```
blog_venv/Scripts/activate

```

Then, let's check the server - 


```
 python manage.py runserver

```


When the server is up and running you will check your home address:

http://127.0.0.1:8000/

and ur home page should show something like:


In [5]:
Image(url= "../img/blog.png")


our about page should look like this:

In [8]:
Image(url= "../img/about.png")


and our post detail page, should look something like this:

In [10]:
Image(url= "../img/post_detail.png")


We know the urls, because we wrote them in our my-blog/blog/urls.py file:

```

urlpatterns = [
    path('', views.home, name='blog-home'),
    path('about/', views.about, name='blog-about'),
    path('post/<int:pk>/', views.post_detail, name='post-detail'),

]


```

meaning the path for home is  http://127.0.0.1:8000/, the path for about is  http://127.0.0.1:8000/about and the path for posts is  http://127.0.0.1:8000/post/<pk> and the pk, being the primary key is usually the number of the post, here:  http://127.0.0.1:8000/post/1. 

If you do not have a blog post yet, log in with your credentials to http://127.0.0.1:8000/admin and create a post.

## Some changes in models. py (Updating the ```Comment``` model)

To enable comment functionality on our blog, we need to update the Comment model in the models.py file. Open the file and add the following fields to the Comment model:

- post: Represents the associated post using a foreign key relationship to the Post model.
- author: Represents the author of the comment using a foreign key relationship to the User model.
- content: Stores the content of the comment as a text field.
- created_date: Stores the date and time when the comment was created using models.DateTimeField(default=timezone.now).


These fields are crucial for capturing the necessary information about a comment.



```

class CommentManager(models.Manager):
    pass


class Comment(models.Model):
    post = models.ForeignKey(Post, related_name='comments', on_delete=models.CASCADE)
    author = models.ForeignKey(User, on_delete=models.CASCADE)
    content = models.TextField()
    created_date = models.DateTimeField(default=timezone.now)
    objects = CommentManager()

    

```

In the ```models.py ``` file, we'll define a new class called  ```CommentManager```, which extends ```models.Manager```. This manager can be used to customize how queries are performed on the Comment model. For now, we won't add any custom methods to keep it simple and move on to the next steps.



Inside the Comment model class, add ```objects = CommentManager()```. This assigns the custom CommentManager to the objects attribute of the Comment model.



We'll implement the __str__ method in the Comment model. This method provides a string representation of a Comment object. Add the following code inside the Comment model class:

````
def __str__(self):
    return str(self.content)


````

Here, we return ```str(self.content)```, which represents the content of the comment.



### The things we need to  do:
    
#### 1. Create CommentForm: 
In our  forms.py file, we will a CommentForm using Django's ModelForm class. This form will allow users to input their comments.

#### 2. Update Post Detail View:
we will update the post_detail view in views.py to handle the rendering of individual post pages along with their associated comments

#### 3. Display Comments in post_detail.html

#### 4. Add Comment Submission Form: 
add the comment submission form to the post_detail.html template
Render the CommentForm and include it in the HTML form. Upon form submission, handle the form data in the post_detail view to create and save the comment to the database.



# Forms in general

In Django, forms are a powerful tool for handling user input, validating data, and rendering HTML forms. They provide an abstraction layer to simplify the process of creating, rendering, and processing forms.

Django forms are defined as Python classes that inherit from the django.forms.Form or django.forms.ModelForm class. Let's explore both types of forms:

1. Form Class (django.forms.Form):

A Django form class (django.forms.Form) is used when you need to create a form that is not directly linked to a model(like an email contact would be).
You define the form fields as class attributes, specifying the field type (CharField, EmailField, IntegerField, etc.) and any additional parameters or validators.

e.g.
```
from django import forms


class MyForm(forms.Form):
    name = forms.CharField(max_length=100)
    email = forms.EmailField()
    age = forms.IntegerField()
```

2. ModelForm Class (django.forms.ModelForm):

A Django model form class (django.forms.ModelForm) is used when you want to create a form based on a model (like our post or comment). It automatically generates form fields based on the model's fields.
You define the form's Meta class within the form class, specifying the associated model and the fields you want to include in the form.


e.g.

```
from django import forms
from .models import MyModel

class MyModelForm(forms.ModelForm):
    class Meta:
        model = MyModel
        fields = ['field1', 'field2']
```


## 1. Create CommentForm: 

Now let's create a new form in the forms.py file called CommentForm. We'ew defining the necessary field for the comment form -the  content.


First create a ```forms.py``` file, inside the blog app, that is in my_blog/my_blog/blog.

In your ```forms.py``` file , create a CommentForm using Django's ModelForm class(https://docs.djangoproject.com/en/4.2/topics/forms/modelforms/). This form will allow users to input their comments. 


The code:


```
from django import forms
from .models import Comment

class CommentForm(forms.ModelForm):
    class Meta:
        model = Comment
        fields = ['content']


```


Once you've defined our form class, we can use it in your views to render the form, validate user input, and process the form data.

To render the form in your template, you pass an instance of the form class to the template context and use the form's properties and methods to render the form fields. e.g.:



## 2. Update Post Detail View:



(or rendering the form)


In your views.py file (inside the blog app), update the post_detail view to handle rendering individual post pages along with their associated comments.We need to add logic to handle the form submission when a user submits a comment. 


What we're doing in the following code is:
- Checking if the request method is POST and validate the comment form. 
- If the form is valid, save the comment with the associated post and author, and redirect back to the post detail page.

 Code:


```


from django.shortcuts import render, get_object_or_404, redirect
#  handles rendering templates and retrieving objects from the database.


from .models import Post, Comment
from .forms import CommentForm

def post_detail(request, pk):
    post = get_object_or_404(Post, pk=pk)
    comments = Comment.objects.filter(post=post)
    
    if request.method == 'POST':
        form = CommentForm(request.POST)
        if form.is_valid():
            comment = form.save(commit=False)
            comment.post = post
            comment.author = request.user
            comment.save()
            return redirect('post-detail', pk=pk)
    else:
        form = CommentForm()
    
    return render(request, 'blog/post_detail.html', {'post': post, 'comments': comments, 'form': form})


```


Here, we fetch the specific post based on its primary key ``` (pk)``` and retrieve the associated comments. We also create an instance of the CommentForm and pass it as context data to the template. If the form is submitted with valid data, we create a new comment, associate it with the post, save it, and redirect back to the post detail page.



We define the post_detail view function, which takes a request object and a pk parameter representing the primary key of the post.

- post = get_object_or_404(Post, pk=pk): Retrieves a Post object with the given primary key (pk) from the database. If the post doesn't exist, it raises a 404 error.

- comments = Comment.objects.filter(post=post): Retrieves all Comment objects associated with the post using a query that filters the comments based on the post.

- then we check if the request method is POST. This condition is used to differentiate between form submissions and initial GET requests.

- form = CommentForm(request.POST): Creates an instance of the CommentForm with the POST data from the request. This binds the form with the submitted data.

- if form.is_valid(): Validates the form, i.e. checks if the submitted data is valid according to the form's field validation rules

- comment = form.save(commit=False): Creates a new Comment object using the form's data but doesn't save it to the database yet

- comment.post = post: sssociates the comment with the post object retrieved earlier.

- comment.author = request.user: Sets the author of the comment as the currently logged-in user (currently yourself - admin username)

- return redirect('post-detail', pk=pk): Redirects the user back to the post detail page after successfully submitting the comment

If the request method is not POST (e.g., it's a GET request), we create an instance of the CommentForm with no data. This renders an empty form.

- the return render function : Renders the post_detail.html template, passing the post, comments, and form as context variables. These variables can be accessed within the template for rendering the post details, comments, and the comment submission form.

- objects = CommentManager(): The objects attribute in the Comment model is assigned the CommentManager class. This allows you to customize the manager for the Comment model, although currently, the CommentManager class is empty (only containing pass).

- def __str__(self):: This method provides a string representation of the Comment object. In this case, it returns the string representation of the comment's content (self.content).



## 3. Display Comments in post_detail.html:



In the post_detail.html template, we need to iterate over the comments and display them

Let's look at whata our post_detail.html looks like currently:


In [11]:
Image(url= "../img/post.png")


Here we're accounting for what a particular article looks like on our post-detail page. Now we would want to see all the comments underneath. 
So, let's add


```
  <hr>
  <h3>Comments</h3>
  {% for comment in comments %}
    <div class="comment">
      <p>{{ comment.content }}</p>
      <p class="author">Comment by {{ comment.author }}</p>
      <hr>
    </div>
  {% empty %}
    <p>No comments yet.</p>
  {% endfor %}


  <h3>Add a Comment</h3>
  <form method="POST">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Submit</button>
  </form>


  <script>
    document.getElementById('comment-form').addEventListener('submit', function(event) {
      event.preventDefault();
  </script>
  

{% endblock content %}


```

inside of our block content and after the article end tag.

In the post_detail.html template, we're adding the HTML code to render the comment form. Using the appropriate template tags, such as {{ form.as_p }} or {{ form.as_table }}, we render the form fields. This step ensures that users can see the comment form on the post detail page.



```<hr>``` - Draws a horizontal line as a separator, if we try it here in the Notebook:<hr>


Then we loop through each comment and display its content and author, separated by a horizontal line. We display th "Add a Comment" heading and render a form with fields for submitting a new comment, including a CSRF token for security and a submit button.  
Then we also add an <action> block, which is actually additional JavaScript code to handle the form submission event. The EventListener listens specifically dor the 'submit' event, which occurs when the form is submitted. The reventDefault()  prevents the default form submission behavior.  It instructs the browser not to perform the usual form submission action, which would cause a page refresh.
 You can use JavaScript frameworks like jQuery or vanilla JavaScript  to submit the form data to the server and process the comment.

#### Digression  
- the CSRF token is a security measure used to protect against Cross-Site Request Forgery attacks. It is necessary to include the CSRF token in forms to verify that the form submission originated from the same website and not from an unauthorized source

Now, when we run our server we should see something like this:

In [4]:
Image(url= "../img/comment.png")


It's not the prettiest - but we do have our post and comment (if we added it). If not, try to add a post through the admin site and a comment here.

# ToDo additional features

Now, we have added the basic functionalities to out blog. We can make posts from the admin site and comments from our blog itself.

But, let's add other cool stuff (optionally):

#### 1. Pagination


#### 2. Adding Topics to Blog Posts


#### 3. Multiple users
